# Generate and Extract Dataset

## Load Datasets

In [1]:
import pandas as pd
import numpy as np

In [2]:
DATASET = []

- **TRUE** : `1`
- **FAKE** : `0`

### [Fake and real news dataset](https://www.kaggle.com/datasets/clmentbisaillon/fake-and-real-news-dataset?select=Fake.csv )

In [3]:
# Download the dataset
%pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d clmentbisaillon/fake-and-real-news-dataset
!unzip "fake-and-real-news-dataset.zip"
!rm "fake-and-real-news-dataset.zip"

Note: you may need to restart the kernel to use updated packages.
mkdir: cannot create directory ‘/home/felipe/.kaggle’: File exists
chmod: cannot access '/root/.kaggle/kaggle.json': Permission denied
100%|██████████████████████████████████████| 41.0M/41.0M [00:02<00:00, 26.5MB/s]
100%|██████████████████████████████████████| 41.0M/41.0M [00:02<00:00, 19.4MB/s]
Archive:  fake-and-real-news-dataset.zip
  inflating: Fake.csv                
  inflating: True.csv                


In [4]:
# Read the dataset of the True
df = pd.read_csv("True.csv")
# Standardize the dataset
df["features"] = df["title"] + " " + df["text"]
df["label"] = 1
df.drop(["title", "text", "subject", "date"], axis=1, inplace=True)
# Add the dataset to the list
DATASET.append(df)
# Show
df.head(5)

,features,label
0,"As U.S. budget fight looms, Republicans flip t...",1
1,U.S. military to accept transgender recruits o...,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,1
3,FBI Russia probe helped by Australian diplomat...,1
4,Trump wants Postal Service to charge 'much mor...,1


In [ ]:
df["features"][7]

In [ ]:
df[df["features"].str.contains('bit.ly')]

In [5]:
# Read the dataset of the True
df = pd.read_csv("Fake.csv")
# Standardize the dataset
df["features"] = df["title"] + " " + df["text"]
df["label"] = 0
df.drop(["title", "text", "subject", "date"], axis=1, inplace=True)
# Add the dataset to the list
DATASET.append(df)
# Show
df.head(5)

,features,label
0,Donald Trump Sends Out Embarrassing New Year’...,0
1,Drunk Bragging Trump Staffer Started Russian ...,0
2,Sheriff David Clarke Becomes An Internet Joke...,0
3,Trump Is So Obsessed He Even Has Obama’s Name...,0
4,Pope Francis Just Called Out Donald Trump Dur...,0


In [ ]:
df["features"][0]

In [ ]:
df[df["features"].str.contains('www.')]

In [6]:
!rm *.csv

### [Fake News Dataset](https://data.mendeley.com/datasets/945z9xkc8d/1)

In [ ]:
import os
!curl -L "https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/945z9xkc8d-1.zip" -o data.zip
!unzip "data.zip" && unzip "Fake News Dataset.zip"
!rm -r *.zip

main_directory = os.path.join(os.getcwd(),"Fake News Dataset")
subdirectories = [x[0] for x in os.walk(main_directory)][1:]
print(main_directory)
dataframes = []
test = []
# Itera a través de los subdirectorios y archivos train.csv
for subdirectory in subdirectories:
    file_path = os.path.join(subdirectory, "train.csv")
    test_path = os.path.join(subdirectory, "test.csv")

    # Verifica si el archivo train.csv existe en el subdirectorio
    if os.path.exists(file_path):
        data = pd.read_csv(file_path)
        dataframes.append(data)

    if os.path.exists(test_path):
        data = pd.read_csv(test_path)
        test.append(data)

# Combina los DataFrames en uno solo
train_data = pd.concat(dataframes, ignore_index=True)
test_data = pd.concat(test,ignore_index=True)
!rm -r "Fake News Dataset"
# Guarda el DataFrame combinado en un archivo train_combined.csv
train_data.to_csv("train.csv", index=False)
test_data.to_csv("test.csv", index=False)

### [GonzaloA/fake_news](https://huggingface.co/datasets/GonzaloA/fake_news)

In [ ]:
# Download the dataset
!curl -L "https://huggingface.co/datasets/GonzaloA/fake_news/resolve/refs%2Fconvert%2Fparquet/default/train/0000.parquet" -o 0.parquet

In [ ]:
# Read the Dataset
df = pd.read_parquet("0.parquet")
# Standardize the dataset
df["features"] = df["title"] + " " + df["text"]
df.drop(["title", "text", "Unnamed: 0"], axis=1, inplace=True)
df.insert(0, "features", df.pop("features"))
# Add the dataset to the list
DATASET.append(df)
# Show
df.head(5)

In [ ]:
!rm *.parquet

### [ErfanMoosaviMonazzah/fake-news-detection-dataset-English](https://huggingface.co/datasets/ErfanMoosaviMonazzah/fake-news-detection-dataset-English)

In [ ]:
# Download the dataset
!curl -L "https://huggingface.co/api/datasets/ErfanMoosaviMonazzah/fake-news-detection-dataset-English/parquet/default/train/0.parquet" -o 0.parquet

In [ ]:
# Read the Dataset
df = pd.read_parquet("0.parquet")
# Standardize the dataset
df["features"] = df["title"] + " " + df["text"]
df.drop(["title", "text", "Unnamed: 0", "date", "subject"], axis=1, inplace=True)
df.insert(0, "features", df.pop("features"))
# Add the dataset to the list
DATASET.append(df)
# Show
df.head(5)

In [ ]:
!rm *.parquet

### [mohammadjavadpirhadi/fake-news-detection-dataset-english](https://huggingface.co/datasets/mohammadjavadpirhadi/fake-news-detection-dataset-english)

In [ ]:
!curl -L "https://huggingface.co/api/datasets/mohammadjavadpirhadi/fake-news-detection-dataset-english/parquet/default/train/0.parquet" -o 0.parquet

In [ ]:
# Read the Dataset
df = pd.read_parquet("0.parquet")
# Standardize the dataset
df["features"] = df["title"] + " " + df["text"]
df.drop(["title", "text", "date", "subject"], axis=1, inplace=True)
df.insert(0, "features", df.pop("features"))
df["label"] = df["label"].apply(lambda x: 1 if x == 0 else 0)

# Add the dataset to the list
DATASET.append(df)
# Show
df.head(5)

In [ ]:
!rm *.parquet

### [pushpdeep/fake_news_combined](https://huggingface.co/datasets/pushpdeep/fake_news_combined)

In [ ]:
!curl -L "https://huggingface.co/api/datasets/pushpdeep/fake_news_combined/parquet/default/train/0.parquet" -o 0.parquet

In [ ]:
# Read the Dataset
df = pd.read_parquet("0.parquet")
# Standardize the dataset
df["features"] = df["text"]
df.drop(["Unnamed: 0", "text"], axis=1, inplace=True)
df.insert(0, "features", df.pop("features"))

# # Add the dataset to the list
DATASET.append(df)
# Show
df.head(5)

In [ ]:
!rm *.parquet

### [argilla/news-fakenews](https://huggingface.co/datasets/argilla/news-fakenews)

In [ ]:
!curl -L "https://huggingface.co/api/datasets/argilla/news-fakenews/parquet/default/train/0.parquet" -o 0.parquet

In [ ]:
# Read the Dataset
df = pd.read_parquet("0.parquet")
# Standardize the dataset
df = df[["text","prediction"]]
df = df[df["text"] != ' ']
df["prediction"] = df["prediction"].apply(lambda x: 1 if x[0]["label"] == 'real' else 0)
df.columns = ["features", "label"]
# Add the dataset to the list
DATASET.append(df)
# Show
df.head(5)

In [ ]:
!rm *.parquet

### Join all the data

In [7]:
df = pd.concat(DATASET, ignore_index=True)
df = df.sample(frac=1).reset_index(drop=True) # Shuffle the dataset
df.to_csv("train.csv", index=False)
del DATASET

## Inspeccionar datos

In [ ]:
df.head(5)

In [ ]:
df.info()

# Visualizacion del Dataset

## Dataset Balanceado

In [ ]:
## Librerias para graficación
import matplotlib.pyplot as plt
import seaborn as sns

# Visualizamos si la data esta balanceada
sns.catplot(x="label", kind="count", color="r", data=df)
plt.title("Distribución de Clasificación")
plt.xlabel("is True")
plt.ylabel("Conteo")

## Word Cloud de los titulares de las *Fake News*

In [ ]:
# importing all necessary modules
from wordcloud import WordCloud, STOPWORDS
comment_words = ""
stopwords = set(STOPWORDS)
# Recorrer el dataframe
for val in df[df["label"] == 0]["features"][:100]:
    # Castear cada palabra a string
    val = str(val)
    # Separamos por palabra
    tokens = val.split()
    # Pasamos a minusculas
    for i in range(len(tokens)):
        tokens[i] = tokens[i].lower()
    comment_words += " ".join(tokens) + " "
wordcloud = WordCloud(
    width=800,
    height=800,
    background_color="white",
    stopwords=stopwords,
    min_font_size=10,
).generate(comment_words)
# Generamos el plot
plt.figure(figsize=(8, 8), facecolor=None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()

In [ ]:
# importing all necessary modules
from wordcloud import WordCloud, STOPWORDS
comment_words = ""
stopwords = set(STOPWORDS)
# Recorrer el dataframe
for val in df[df["label"] == 1]["features"][:100]:
    # Castear cada palabra a string
    val = str(val)
    # Separamos por palabra
    tokens = val.split()
    # Pasamos a minusculas
    for i in range(len(tokens)):
        tokens[i] = tokens[i].lower()
    comment_words += " ".join(tokens) + " "
wordcloud = WordCloud(
    width=800,
    height=800,
    background_color="white",
    stopwords=stopwords,
    min_font_size=10,
).generate(comment_words)
# Generamos el plot
plt.figure(figsize=(8, 8), facecolor=None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()

# Pre-Procesado de los datos para NLP

In [ ]:
df = pd.read_csv("train.csv")

# shuffle data
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
# df.columns = ["features", "label"]

In [51]:
df["features"][18043]

'fed republican threaten third party option trump nominee saying long time republican party verge splitting right sort always assumed establishment would control party iron fist tea party white supremacist would abandon gop form white power party rise donald trump look like establishment going abandon party run candidate raving sociopathspurred donald trump mounting victory small influential growing group conservative leader calling thirdparty option spare voter wrenching general election choice republican consider completely unacceptable hillary clintonwhile gained intense popularity right mr trump alienated key bloc republican coalition slashandburn campaign many initial refusal last weekend disavow endorsement david duke white supremacist breaking point breaking point as problem conservative leader trump racism overt nature republican relied white resentment year win election keep concealed lest get publicly torn apart party old white racist corporate medium happy promote fiction tr

In [50]:
df[df["features"].str.contains('xax')]

,features,label
18043,fed republican threaten third party option tru...,0


In [42]:
test = processing_text(df["features"][3])

In [43]:
test

'bernie sander crashed greedy drug company stock one tweet appears former democratic presidential candidate senator bernie sander tanked drug company stock one tweet tweet directed towards ariad pharmaceutical ariad owns treatment chronic myeloid leukemia callediclusig ariad raised price drug four time recent historythe last time raised price drug went percent cost dollar month nearly year price hike doubled ariad sale total million dollar one quarterthe tweet sent said drug corporation greed unbelievable ariad raised price leukemia drug almost year drug corporation greed unbelievable ariad raised price leukemia drug almost year bernie sander sensanders october tweet also included link article company continual price increasesbloomberg report sander likely one staffer sent tweet ariad stock value dropped much percent one worst day company yearat time writing tweet appear gone viral compared many senator social medium posting retweets like though may change attention story sander help m

In [41]:
import re
'''
* processing_text
* @param texto str
* @return processed_feature str
'''
def processing_text(texto):
    
    # Expresión regular para encontrar enlaces
    regex_enlaces = re.compile(r'https?://\S+|www.\S+', re.IGNORECASE)

    # Elimina los enlaces del texto
    processed_feature = regex_enlaces.sub('', texto)

    # 1. Limpiar textos
    processed_feature = re.sub(r'[^a-zA-Z0-9 ]', '', str(processed_feature)) # Remover con un expresión regular carateres especiales (no palabras).
    # processed_feature = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_feature) # Remover ocurrencias de caracteres individuales
    processed_feature = re.sub(r'[0-9]+', ' ', processed_feature) #  Remover números (Ocurrencias muy esporádicas en nuestro dataset)
    processed_feature = processed_feature.lower() # Pasar todo el texto a minúsculas
    
    # 2. Stop words & lemmatization
    from nltk.stem import WordNetLemmatizer
    from nltk.corpus import stopwords

    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    processed_feature = processed_feature.split()
    processed_feature = ' '.join([lemmatizer.lemmatize(word) for word in processed_feature if word not in stop_words])

    processed_feature= re.sub(r'\b[a-zA-Z]\b', '', processed_feature) # Remover ocurrencias de caracteres individuales
    processed_feature = re.sub(' +', ' ', processed_feature) # Simplificar espacios concecutivos a un único espacio entre palabras
    return processed_feature

In [44]:
df["features"] = df["features"].apply(processing_text)

# Save the data

In [45]:
df.to_json("features.json",orient='records', lines=True)